In [1]:
pwd()

'/home/mb/MEGAsync/academic_work/projects/geometric_model_polycrystals/code/PyAPD/notebooks/extensions'

In [2]:
# SETUP:

# reloading modules if needed
from importlib import reload

# first import the whole package
import PyAPD

# then load all the functions in the .py files in the .../PyAPD/ directory:
from PyAPD import *

import copy

torch.set_printoptions(precision=10) # show more digits

device = "cuda" if torch.cuda.is_available() else "cpu"
dt = torch.float64
torch.set_default_dtype(dt)

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [3]:
from pykeops.torch import LazyTensor

In [4]:
import pykeops

In [5]:
file = "../../data/2D_Tata_Steel_paper_example/sample_lc_steel_grain_file_final.txt"
seed_id = (0,1)
volumes_id = (5)
orientation_id = (2,3,4)

X, A, TV, EBSD = load_setup_from_EBSD_data_2D(file = file,
                                             seed_id = seed_id,
                                             volumes_id = volumes_id,
                                             orientation_id = orientation_id,
                                             normalise_matrices = True,
                                             angle_in_degrees = False,
                                             device = device,
                                             dt = dt)
X = X.to(device,dtype=dt)
A = A.to(device,dtype=dt)
TV = TV.to(device,dtype=dt)
N, D = X.shape

In [6]:
# Manually enter the correct domain size:
dom_x = [-999.02675,0.49975]
dom_y = [-901.0003,0.5003]
domain = torch.tensor([dom_x,dom_y])

In [7]:
# Compute total area and show largest and smallest grain:
total_area = TV.sum()
print("Largest grain: ", torch.max(TV).cpu().numpy(), "Smallest grain: ",torch.min(TV).cpu().numpy())

Largest grain:  2073.79661128761 Smallest grain:  11.000898081641


In [8]:
# Construct an APD system:
apd1 = apd_system(D=2,
                  domain = domain,
                  X = X,
                  As = A,
                  heuristic_W = True, # the default set of weights is the heuristic guess
                  target_masses = TV/total_area,
                  dt = torch.float64,
                  device = device,
                  pixel_size_prefactor = 1.0,
                  error_tolerance = 0.01
                  )

In [9]:
# Load EBSD pixelated data: positions of pixels and grain assignment (from MTEX):
file = "../../data/2D_Tata_Steel_paper_example/grainIDs_final.txt"
EBSD_pixel_data = torch.from_numpy(np.loadtxt(file)).to(device=apd1.device, dtype=apd1.dt)

In [9]:
# preparation
eps = 0.1

Y = EBSD_pixel_data[:,0:2]
II = EBSD_pixel_data[:,2].int()
y = LazyTensor(Y.view(1, len(Y), 2))


In [10]:
#normal_sumz = (-sD_ij/eps).sum()

NameError: name 'sD_ij' is not defined

In [169]:
#apd1.X[(II-1)].shape

torch.Size([896361, 2])

In [170]:
#D_ij = ((y - apd1.x) | apd1.a.matvecmult(y - apd1.x)) - apd1.w

In [171]:
#D_ij.shape

(4587, 896361)

In [172]:
#proper_sumz = (-D_ij/eps).logsumexp(dim=0)

[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.


In [173]:
#proper_sumz.shape

torch.Size([896361, 1])

In [174]:
#proper_sumz.sum(dim=0)

tensor([5.9737688596e+08])

In [176]:
#total_sum = normal_sumz - proper_sumz.sum(dim=0)

In [177]:
#total_sum

tensor([-40965014.6862816811])

In [11]:
def objective_function(X,A,W):
    dims = A.shape
    MV = torch.einsum('bij,bj->bi', torch.index_select(A,0,II-1), torch.index_select(X, 0, II-1) - Y)
    sD_ij = torch.einsum('bj,bj->b',MV,torch.index_select(X,0,II-1)-Y) - torch.index_select(W, 0, II-1)
    first_sum = (-sD_ij/eps).sum()
    
    a = LazyTensor(A.view(dims[0], 1, dims[1] * dims[2]))
    w = LazyTensor(W.view(dims[0],1,1))
    x = LazyTensor(X.view(dims[0], 1, dims[1]))
    D_ij = ((y - x) | a.matvecmult(y - x)) - w
    second_sum = (-D_ij/eps).logsumexp(dim=0).sum(dim=0)
    return first_sum - second_sum


In [12]:
objective_function(apd1.X,apd1.As,apd1.W)

[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.


tensor([-40965014.6862816811])